In [46]:
import os
import torch
import glob
import numpy as np

In [142]:
directory = '/home/watson/Documents/CIFAR/cifar-10-python/cifar-10-batches-py'
data_prefix = 'data'
test_prefix = 'test'
num_channels = 3

In [163]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def changeDimension(x):
    
    assert isinstance(x,list),'x must be list type'
    np_x = np.array(x)
    print(np_x.shape)
    sp = np_x.shape
    size_per_channel = sp[-1]/num_channels
    len_per_side = int(np.sqrt(size_per_channel))
    if len(sp) == 2:
        new_array = np.reshape(np_x,(sp[0]*sp[1]))
    if len(sp) == 3:
        new_array = np.reshape(np_x,(sp[0]*sp[1],num_channels,len_per_side,len_per_side))

    

    return new_array



In [ ]:
def center_crop(x,c_x=28,c_y=28):
    sp = x.shape

    start_x = (len_per_side - c_x)//2
    stop_x = start_x + c_x
    start_y = (len_per_side-c_y)//2
    stop_y = start_y + c_y
    image_three = []
    for i in range(sp[0]):
        for j in range(num_channels):
            image = 
            
    return y

def preprocessing(x):
    normalize()
    center_crop()
    whitening()
    return x

In [164]:
training_files = glob.glob(directory+os.sep+data_prefix+'*')
test_files = glob.glob(directory+os.sep+test_prefix+'*')

In [165]:
training_raw_images = []
training_labels = []

test_raw_images = []
test_labels = []

In [166]:
for file in training_files:
    batch_dict = unpickle(file)
    training_raw_images.append(batch_dict[b'data'])
    training_labels.append(batch_dict[b'labels'])
for file in test_files:
    batch_dict = unpickle(file)
    test_raw_images.append(batch_dict[b'data'])
    test_labels.append(batch_dict[b'labels'])

In [167]:
training_raw_images = changeDimension(training_raw_images)
training_labels = changeDimension(training_labels)
test_raw_images = changeDimension(test_raw_images)
test_labels = changeDimension(test_labels)

(5, 10000, 3072)
(5, 10000)
(1, 10000, 3072)
(1, 10000)


In [169]:
print(training_labels.shape)

(50000,)


In [141]:
np.sqrt(1024)

32.0